<a href="https://colab.research.google.com/github/QuinnG17/Financial-Programs/blob/main/QUANT_CONNECT_INCREMENTAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%sql create or replace table alpha_two.ALL_DATE_METRICS as select * from final_output

In [ ]:

api_key = '28c4dda990c8903ccba29687b06969a4120680242b523193d087919db39984b4'

# Get timestamp
timestamp = str(int(time.time()))
time_stamped_token = api_key + ':' + timestamp


# Get hased API token
hashed_token = hashlib.sha256(time_stamped_token.encode('utf-8')).hexdigest()
authentication = "{}:{}".format(222110, hashed_token)
api_token = base64.b64encode(authentication.encode('utf-8')).decode('ascii')


# Create headers dictionary.
headers = {
    'Authorization': 'Basic %s' % api_token,
    'Timestamp': timestamp
}


base_url = 'https://www.quantconnect.com/api/v2/backtests/read/BacktestList'
project_id = 14480263


# Create POST Request with headers (optional: Json Content as data argument).
response = requests.post(base_url,
                         data = {
                             "projectId": project_id
                                 },
                         headers = headers)


backtestList = response.json()
backtests = []
backtestTrades = []
base_url = 'https://www.quantconnect.com/api/v2/backtests/read/Trade'


for backtest in backtestList['backtests']:
    backtests.append(backtest['backtestId'])

#print(backtests[0])


#for backtest in backtests[0]:
response = requests.post(base_url,
                         data = {
                             #"start": 0,
                             #"end": 0,
                             "projectId": project_id,
                             "backtestId": backtests[201]
                                 },
                         headers = headers)


results = response.json()
trades = results['backtest']['charts']['Strategy Equity']['Series']['Daily Performance']['Values']
symbol = results['backtest']['totalPerformance']['ClosedTrades'][0]['Symbol']['Permtick']
start_date = pd.to_datetime(results['backtest']['backtestStart'])
end_date = pd.to_datetime(results['backtest']['backtestEnd'])


df = pd.DataFrame(trades)
df = df.transpose()
df.columns = df.iloc[0]
df = df.drop(df.index[0])
df['Symbol'] = symbol

df = df.reset_index()
df = df.drop(df.columns[[0, 1]], axis=1)
col_name = 'Symbol'
first_col = df.pop(col_name)
df.insert(0, col_name, first_col)

# Create a date range with the desired frequency (1 day in this case)
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# Set the date range as the column names of the DataFrame
df.columns = ['Symbol'] + list(date_range)

valid_backtests = []



for backtest in backtests:

    response = requests.post(base_url,
                             data = {
                                 #"start": 0,
                                 #"end": 0,
                                 "projectId": project_id,
                                 "backtestId": backtest
                                     },
                             headers = headers)

    results = response.json()

    if pd.to_datetime(results['backtest']['backtestEnd']) < daily_returns['Date'].iloc[-1]:
        continue

    if len(results['backtest']['charts']) == 0:
        continue
    if len(results['backtest']['charts']['Strategy Equity']) == 0:
        continue
    if 'Series' not in results['backtest']['charts']['Strategy Equity'].keys():
        continue


    trades = results['backtest']['charts']['Strategy Equity']['Series']['Daily Performance']['Values']

    if len(trades) == 0:
        continue


    if len(results['backtest']['totalPerformance']['ClosedTrades']) == 0:
        continue


    symbol = results['backtest']['totalPerformance']['ClosedTrades'][0]['Symbol']['Permtick']
    trades = results['backtest']['charts']['Strategy Equity']['Series']['Daily Performance']['Values']
    start_date = pd.to_datetime(results['backtest']['backtestStart'])
    end_date = pd.to_datetime(results['backtest']['backtestEnd'])
    df1 = pd.DataFrame(trades)
    df1 = df1.transpose()
    df1.columns = df1.iloc[0]
    df1 = df1.drop(df1.index[0])
    df1['Symbol'] = symbol


    df1 = df1.reset_index()
    df1 = df1.drop(df1.columns[[0, 1]], axis=1)
    col_name = 'Symbol'
    first_col = df1.pop(col_name)
    df1.insert(0, col_name, first_col)

    # Create a date range with the desired frequency (1 day in this case)
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')


    # Set the date range as the column names of the DataFrame
    if len(list(date_range)) + 1 != len(df1.columns):
        continue

    df1.columns = ['Symbol'] + list(date_range)

    df = df.append(df1, sort=False).fillna(0)

    if len(df) == 10:
       df = df

    print(len(df) / len(backtests))


df = df.reset_index()
col_name = 'Symbol'
first_col = df.pop(col_name)
df.insert(0, col_name, first_col)
df = df.set_index('Symbol')


In [ ]:
df = df.drop_duplicates()
df = df.drop(columns='index')
df = df.T
df = df.reset_index(drop=False)
df = df.rename(columns=lambda x: x.strip())
df = df.rename(columns={'index': 'Date'})

df.iloc[:, 1:] = df.iloc[:, 1:].astype(float)
df = df.iloc[:, ~df.columns.duplicated(keep='last')]


In [ ]:
df = df.set_index('Date')
df = df[df.index > daily_returns['Date'].iloc[-1]]


In [ ]:
df = df.reset_index()


In [ ]:
full = pd.concat([daily_returns, df]).fillna(0)


In [ ]:
from pyspark.sql import SparkSession

df.iloc[:, 1:] = df.iloc[:, 1:].astype(float)

spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")

# Create a SparkSession
spark2 = SparkSession.builder.appName('pandas-to-databricks').getOrCreate()

final_daily_changes = spark2.createDataFrame(full)

final_daily_changes.createOrReplaceTempView('final_daily_changes')

In [ ]:
%sql create or replace table alpha_two.ALL_DAILY_RETURNS_BACKUP as select * from alpha_two.ALL_DAILY_RETURNS

In [ ]:
%sql create or replace table alpha_two.ALL_DAILY_RETURNS as select * from final_daily_changes